In [2]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
2,236,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,20/01/2023,20/01/2023 12:00 HORAS,PASCO/PASCO/TINYAHUARCO,KM 117,(SECTOR YANAMATE) PARO INDEFINIDO SOLICITANDO ...,-10.731252,...,-10.731252+-76.251696,2023 12:00 HORAS+01+20+12:00,0,0,COMISARIA COLQUIJIRCA,HUMANO,PE-3N,LONGITUDINAL DE LA SIERRA NORTE,SI,interrumpido
3,235,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,20/01/2023,20/01/2023 12:00 HORAS,PASCO/PASCO/NINACACA,KM 92,(SECTOR NINACACA) PARO INDEFINIDO SOLICITANDO ...,-10.87051434,...,-10.87051434+-76.11033301,2023 12:00 HORAS+01+20+12:00,0,0,COMISARIA COLQUIJIRCA,HUMANO,PE-3N,LONGITUDINAL DE LA SIERRA NORTE,SI,interrumpido
4,212,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,10/01/2023,20/01/2023 12:00 HORAS,MADRE DE DIOS/TAHUAMANU/IBERIA,KM 597,GRUPO DE PERSONAS PROTESTAN SOLICITANDO NUEVA...,-11.4068843,...,-11.4068843+-69.4784365,2023 12:00 HORAS+01+20+12:00,0,0,CEOPOL,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
5,223,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,06/01/2023,20/01/2023 12:00 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 511+800,(PUENTE MAVILA) PARO INDEFINIDO SOLICITANDO NU...,-11.931332,...,-11.931332+-69.11748,2023 12:00 HORAS+01+20+12:00,0,0,CEOPOL,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
6,225,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,08/01/2023,20/01/2023 12:00 HORAS,MADRE DE DIOS/TAMBOPATA/TAMBOPATA,KM 402,(SAN BERNARDO) PARO INDEFINIDO SOLICITANDO NUE...,-12.6898325,...,-12.6898325+-69.4042951,2023 12:00 HORAS+01+20+12:00,0,0,CEOPOL,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,205,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,11/01/2023,20/01/2023 12:00 HORAS,LA LIBERTAD/VIRU/VIRU,KM 528,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-8.382992,...,-8.382992+-78.855957,2023 12:00 HORAS+01+20+12:00,0,1,CONCESIONARIA AUTOPISTA DEL NORTE,HUMANO,PE-1N,LONGITUDINAL DE LA COSTA NORTE,SI,interrumpido
117,204,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,16/01/2023,20/01/2023 12:00 HORAS,LA LIBERTAD/VIRU/VIRU,KM 513,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-8.449496593,...,-8.449496593+-78.73499237,2023 12:00 HORAS+01+20+12:00,19,4,CONCESIONARIA AUTOPISTA DEL NORTE,HUMANO,PE-1N,LONGITUDINAL DE LA COSTA NORTE,SI,interrumpido
118,203,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,16/01/2023,20/01/2023 12:00 HORAS,LA LIBERTAD/VIRU/CHAO,KM 500,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-8.546018905,...,-8.546018905+-78.67488073,2023 12:00 HORAS+01+20+12:00,15,8,CONCESIONARIA AUTOPISTA DEL NORTE,HUMANO,PE-1N,LONGITUDINAL DE LA COSTA NORTE,SI,interrumpido
119,326,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,19/01/2023,20/01/2023 12:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 391,SECTOR HUIPOCA. PARO INDEFINIDO SOLICITANDO NU...,-8.9903469087213,...,-8.9903469087213+-75.452262878851,2023 12:00 HORAS+01+20+12:00,0,0,UD UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
